In [ ]:
!git clone https://github.com/chickowar/6-sem-CV-dz-4

In [ ]:
%cd 6-sem-CV-dz-4/

In [ ]:
import sys
sys.path.append('./src') # чтобы он видел модуль src

In [ ]:
from src.datasets import get_mnist, get_svhn
from src.dann_model import FeatureExtractor, LabelClassifier, DomainDiscriminator
from src.dann_train import train_dann
from src.train_eval import evaluate
import torch
import torch.optim as optim
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Загрузка датасетов и модельки

In [ ]:
source_loader = get_mnist(train=True)
target_loader = get_svhn()
source_test = get_mnist(train=False)
target_test = get_svhn()

In [ ]:
F = FeatureExtractor()
C = LabelClassifier()
D = DomainDiscriminator()

# Обучение

In [ ]:
def alpha_schedule(epoch, step, max_epoch=10, max_step=500):
    p = float(epoch * max_step + step) / (max_epoch * max_step)
    return 2. / (1. + torch.exp(-10 * p)) - 1

In [ ]:
train_dann(F, C, D, source_loader, target_loader, device,
           epochs=10, alpha_schedule=lambda e, i: alpha_schedule(e, i))

# Оценка

In [ ]:
print("Eval on Source (MNIST):")
evaluate(nn.Sequential(F, C), source_test, device)

In [ ]:
print("Eval on Target (SVHN):")
evaluate(nn.Sequential(F, C), target_test, device)